# Causal gene regulatory network reconstruction

Cells continuously monitor their internal and external environment and calculate the amount at which each type of protein is needed. This information-processing function is carried out by [gene regulatory networks (GRNs)](https://en.wikipedia.org/wiki/Gene_regulatory_network), which control the rate of production of each protein. Two important classes of regulatory molecules in GRNs are [transcription factors](https://en.wikipedia.org/wiki/Transcription_factor) and [microRNAs](https://en.wikipedia.org/wiki/MicroRNA).

In this tutorial we will use causal inference to reconstruct GRNs from [mRNA](https://en.wikipedia.org/wiki/Messenger_RNA) and [microRNA](https://en.wikipedia.org/wiki/MicroRNA) expression data from the [GEUVADIS study](https://www.nature.com/articles/nature12531). In particular, we will use genetic variants as causal instruments and the [BioFindr software](https://github.com/tmichoel/BioFindr.jl) for model selection, as introduced in the [blessing of dimensionality notebook](2_blessing_of_dimensionality.ipynb).

## Setup the environment

In [1]:
using DrWatson
quickactivate(@__DIR__)

In [2]:
using DataFrames
using Arrow
using Gadfly
using Compose
using BioFindr

## Load data

This tutorial uses [preprocessed data files](https://github.com/lingfeiwang/findr-data-geuvadis) from the [GEUVADIS study](https://doi.org/10.1038/nature12531):